<a href="https://colab.research.google.com/github/rvargas42/TensorFlow_DeepLearningAI_Course/blob/main/C1/C1_W4_ConvNets_Preprocessing_Batches_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Nets with real world case


> Image Data Generator: you point it at a directory and it will generate the images in subdirectories into training and testing folders.



In [18]:
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip

!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip

--2024-03-07 11:26:25--  https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.115.207, 172.253.122.207, 172.253.63.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.115.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘horse-or-human.zip.1’

horse-or-human.zip. 100%[===================>] 142.65M   183MB/s    in 0.8s    

2024-03-07 11:26:25 (183 MB/s) - ‘horse-or-human.zip.1’ saved [149574867/149574867]

--2024-03-07 11:26:25--  https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.115.207, 172.253.122.207, 172.253.63.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.115.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1

In [24]:
import zipfile

# Unzip the dataset
local_zip = './horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./horse-or-human')
zip_ref.close()

local_zip = './validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./validation-horse-or-human')
zip_ref.close()

In [25]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator


print(tf.__version__)

2.15.0


## 1. Image Generator

In [26]:
import os

# Directory with our training horse pictures
train_horse_dir = os.path.join('./horse-or-human/horses')

# Directory with our training human pictures
train_human_dir = os.path.join('./horse-or-human/humans')

#Directory with validation for horses
validation_horse_dir = os.path.join('./validation-horse-human/horses')

#Directory with validation for humans
validation_horse_dir = os.path.join('./validation-horse-human/humans')

In [30]:
train_dir = "./horse-or-human"
train_data_generator = ImageDataGenerator(rescale=1./255)
train_generator = train_data_generator.flow_from_directory(
    train_dir,
    target_size=(300,300), #this is the size at which the images are resized
    batch_size=128, #this is the number of images processed
    class_mode='binary' #as we are building a binary classifier
)

val_dir = "./validation-horse-or-human"
val_data_generator = ImageDataGenerator(rescale=(1./255))
val_generator = val_data_generator.flow_from_directory(
    val_dir,
    target_size=(300,300),
    batch_size=32,
    class_mode='binary'
)

Found 1283 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


## 2. Designing the Model

In [34]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        # The second convolution
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        # The third convolution
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        # The fourth convolution
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        # The fifth convolution
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        keras.layers.Flatten(),
        keras.layers.Dense(512,activation='relu'),
        keras.layers.Dense(1, activation='sigmoid') #sigmoid activation function as it is a binary classifier
    ]
)

In [35]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 149, 149, 16)      0         
 ng2D)                                                           
                                                                 
 conv2d_11 (Conv2D)          (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 73, 73, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_12 (Conv2D)          (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 35, 35, 64)       

# 3. Compiling and fitting the optimizer and loss

In [36]:
from tensorflow.keras.optimizers import RMSprop

model.compile(
    loss='binary_crossentropy',
    optimizer=RMSprop(learning_rate=0.001),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=8, #there are 1024 images in the training dir so we set the necessary steps per epoch  to cover all images 8 * 128 = 1042
    epochs= 15,
    validation_data=val_generator,
    validation_steps=8,
    verbose=1
)

Epoch 1/15
8/8 [==============================] - 89s 11s/step - loss: 0.6524 - accuracy: 0.5740 - val_loss: 0.9757 - val_accuracy: 0.5000
Epoch 2/15
8/8 [==============================] - 97s 12s/step - loss: 0.6855 - accuracy: 0.5645 - val_loss: 0.5459 - val_accuracy: 0.6211
Epoch 3/15
8/8 [==============================] - 85s 10s/step - loss: 0.4983 - accuracy: 0.8120 - val_loss: 0.2381 - val_accuracy: 0.9297
Epoch 4/15
8/8 [==============================] - 99s 12s/step - loss: 0.4249 - accuracy: 0.8109 - val_loss: 0.4528 - val_accuracy: 0.8633
Epoch 5/15
8/8 [==============================] - 86s 11s/step - loss: 0.8398 - accuracy: 0.7842 - val_loss: 0.4159 - val_accuracy: 0.9414
Epoch 6/15
8/8 [==============================] - 89s 11s/step - loss: 0.5912 - accuracy: 0.7720 - val_loss: 0.4193 - val_accuracy: 0.9492
Epoch 7/15
8/8 [==============================] - 87s 11s/step - loss: 0.4398 - accuracy: 0.8899 - val_loss: 0.3896 - val_accuracy: 0.8594
Epoch 8/15
8/8 [===========

## 4. Predicting

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

uploaded = files.upload()

for fn in uploaded.keys():

  # predicting images
  path = '/content/' + fn
  img = load_img(path, target_size=(300, 300))
  x = img_to_array(img)
  x /= 255
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])

  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")